Updated 1/20/2021 by Johnson Kuan

# Audio Classification with Cleanlab and LightGBM

In this tutorial, we will use Cleanlab to find potential label errors in the Spoken Digit dataset (it's like MNIST for audio). The dataset contains 2,500 audio clips with english pronunciations of the digits 0 to 9. 

**High level overview of what we'll do in this tutorial:**

- Extract features from audio clips (.wav files) using a pre-trained model from Hugging Face trained on the [VoxCeleb](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/) speech dataset.

- Train a cross-validated LightGBM model using the extracted features.

- Generate cross-validated predicted probabilities to use as input into Cleanlab.

- Generate a list of audio clips with potential label errors.

**Data:** https://www.tensorflow.org/datasets/catalog/spoken_digit

**Pre-Trained Model:** https://huggingface.co/speechbrain/spkrec-xvect-voxceleb

**Cleanlab Repo:** https://github.com/cleanlab/cleanlab

*Data-Centric AI* 

*Data-Centric ML*

## **1. Install dependencies and import libraries**

To get started, let's run below to install additional packages for your Colab environment.

In [7]:
%%capture

%pip install cleanlab speechbrain tensorflow-io tensorflow lightgbm pandas matplotlib wget

Note: you may need to restart the kernel to use updated packages.


Next, let's import all the packages you need for this tutorial. Note that we also set a random seed to make experiments reproducible.

In [2]:
import cleanlab
import tensorflow as tf
import tensorflow_io as tfio
import torchaudio
from speechbrain.pretrained import EncoderClassifier
import lightgbm as lgb
from sklearn.model_selection import cross_val_predict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from IPython import display

np.random.seed(12345)

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


## **2. Download dataset**

We can run below to download the dataset directly into your Colab workspace. Once the download is complete, you should be able to see it on the left menu bar in the "Files" tab.

In [13]:
import wget
import tarfile

url = "https://github.com/Jakobovski/free-spoken-digit-dataset/archive/v1.0.9.tar.gz"
filename = wget.download(url)

dir = 'spoken_digits'

os.mkdir(dir)

tar = tarfile.open(filename)
tar.extractall(dir)
tar.close()

The audio data (.wav files) is in the "recordings" folder. Let's run below to get a list of all the file names.

Note that the label (digits from 0 to 9) is indicated in the prefix of the file name (e.g. "6_nicolas_32.wav" has the label 6)

In [14]:
DATA_PATH = "spoken_digits/free-spoken-digit-dataset-1.0.9/recordings/"

# Get list of .wav file names
file_paths = []
for (dirpath, dirnames, filenames) in os.walk(DATA_PATH):
    file_paths += [os.path.join(dirpath, file) for file in filenames if file.endswith(".wav")]

# Check out first 3 files
file_paths[:3]

['spoken_digits/free-spoken-digit-dataset-1.0.9/recordings/0_george_0.wav',
 'spoken_digits/free-spoken-digit-dataset-1.0.9/recordings/0_george_1.wav',
 'spoken_digits/free-spoken-digit-dataset-1.0.9/recordings/0_george_10.wav']

## **3. Use pre-trained audio classifier from Huggingface**

Next, let's instantiate an audio feature extractor using the `EncoderClassifier()` class. Note that we pass in the name of the pre-trained model as an argument ("speechbrain/spkrec-xvect-voxceleb"). We will use a model that has been pre-trained on the [VoxCeleb](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/) speech dataset.

In [17]:
# Instantiate audio classifier
feature_extractor = EncoderClassifier.from_hparams(
  "speechbrain/spkrec-xvect-voxceleb",
  # GPU is optional
  # to enable GPU in Colab, go to: edit -> notebook settings -> choose GPU accelerator)
  # run_opts={"device":"cuda"}
)

OSError: [WinError 1314] A required privilege is not held by the client: 'C:\\Users\\lok_w\\.cache\\huggingface\\hub\\5eabefcc6a51c24ac6347df607707a3f297c557549905ac98cdeabc0f86905a8.6547fa3d76c979e5bda1b1690be6db4605abf9c528b8fc58b823cec1becf0fca' -> 'pretrained_models\\EncoderClassifier--7997810228735713788\\hyperparams.yaml'

## **4. Explore the data by playing some audio clips**

Before training the model, let's listen to some of the audio clips. 

We can use the utility function below to process the .wav file so we can listen to it in this code notebook.

In [ ]:
# Utility function for loading audio files and making sure the sample rate is correct.
@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

Click the play button below to listen to the .wav file. 

Feel free to change the `wav_file_name_example` variable below to listen to other audio clips.

In [ ]:
# Single .wav file example
wav_file_name_example = "spoken_digits/free-spoken-digit-dataset-1.0.9/recordings/7_jackson_43.wav"

wav_file_example = load_wav_16k_mono(wav_file_name_example)

# Play the audio file.
display.Audio(wav_file_example, rate=16000)

## **5. Extract features from audio**

Next, run below to use our pre-trained model to extract features (aka embeddings) from the audio clips.

In [ ]:
# Create dataframe with .wav file names
df = pd.DataFrame(file_paths, columns=["wav_audio_file_path"])
df["label"] = df.wav_audio_file_path.map(lambda x: int(Path(x).parts[-1].split("_")[0]))

df.head(3)

In [ ]:
# Feature extractor
def extract_audio_embeddings(model, wav_audio_file_path: str) -> tuple:

    # Transform audio file into a tensor
    signal, fs = torchaudio.load(wav_audio_file_path)

    # Extract features (aka embeddings)
    embeddings = model.encode_batch(signal)

    return embeddings

In [ ]:
# Extract audio embeddings
embeddings_list = []
for i, file_name in enumerate(df.wav_audio_file_path): # for each .wav file name
    embeddings = extract_audio_embeddings(feature_extractor, file_name)
    embeddings_list.append(embeddings.cpu().numpy())

embeddings_array = np.squeeze(np.array(embeddings_list))

Now we have our features in an array! 

Each row in the array corresponds to an audio clip. We're now able to represent an audio clip as a 512-dimensional feature vector!

In [ ]:
embeddings_array.shape

## **6. Train LightGBM model using Cross Validation to generate predicted probabilties**

Run below to train a LightGBM model to generate cross-validated predicted probabilities.

Note that we need to use the LightGBM `Dataset` class to make sure it is in the proper format for our `LGBMClassifier`.

In [ ]:
data = lgb.Dataset(embeddings_array, label=df.label.values)

For convenience, we can use the scikit-learn's `cross_val_predict` function to run k-fold cross-validation and generate cross-validated predicted probabilities.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html

This means that for each audio clip which belongs to a partition of the data (due to k-fold procedure), the predicted probabilities are scored by a model that was trained on the other partitions. This is also known as out-of-sample predictions.

In [ ]:
model = lgb.LGBMClassifier(
    n_estimators=1000,
    eval_metric="multi_logloss",
    verbose=1,
)

# Generate cross-validated predicted probabilities for each input data point
# Note: Cleanlab requires predictions to be out-of-sample (e.g. via cross-validation)
cv_pred_probs = cross_val_predict(
    estimator=model,
    X=embeddings_array,
    y=df.label.values,
    cv=5,
    method="predict_proba"
)

In [ ]:
cv_pred_probs.shape

Run below to get the cross-validation accuracy.

In [ ]:
cv_accuracy = (cv_pred_probs.argmax(axis=1) == df.label.values).mean()

print(f"Cross-Validation Accuracy: {cv_accuracy}")

## **7. Run Cleanlab to find label errors**

Now let's run Cleanlab to find label errors! 

Use the `get_noise_indices()` function below to get an ordered list of indices corresponding to the audio clips with potential label error.

In [ ]:
# Generate an ordered list of indices corresponding to the audio clips with potential label error
ordered_label_errors = cleanlab.pruning.get_noise_indices(
    s=df.label.values,
    psx=cv_pred_probs,
    sorted_index_method="normalized_margin", # Orders label errors
 )

In [ ]:

ordered_label_errors

Run below to get the candidates to inspect for label error.

In [ ]:
print("Candidates to inspect for label error")
df.iloc[ordered_label_errors]

## **8. Listen to these examples of label errors that were found**

After inspecting the list of candidates, you should find that there are indeed label errors. 

Listen to the audio clips below of label errors that were found.

Given label is the digit **6** but sounds more like **8**

In [ ]:
# Single .wav file example
wav_file_name_example = "spoken_digits/free-spoken-digit-dataset-1.0.9/recordings/6_yweweler_14.wav"
wav_file_example = load_wav_16k_mono(wav_file_name_example)

# Play the audio file.
display.Audio(wav_file_example, rate=16000)

Given label is the digit **7** but sounds more like **0**

In [ ]:
# Single .wav file example
wav_file_name_example = "spoken_digits/free-spoken-digit-dataset-1.0.9/recordings/7_nicolas_43.wav"
wav_file_example = load_wav_16k_mono(wav_file_name_example)

# Play the audio file.
display.Audio(wav_file_example, rate=16000)